In [ ]:
import multiprocessing as mp
import numpy as np
import os
import threading

In [ ]:
print("# CPU: {}".format(mp.cpu_count()))

# La classe `Process`

In [ ]:
numeri = np.arange(4000)

In [ ]:
import time

In [ ]:
def conta_numeri_primi(numeri, dormi={}, verboso=False):

    if verboso:
        pid = os.getpid()
        pname = mp.current_process().name
        tname = threading.current_thread().name
        print("PID: {}, Processo: {}, Thread: {}".format(pid, pname, tname))
    
    c = 0
    
    for n in numeri:

        if n in dormi:
            time.sleep(dormi[n])

        if (n >= 2) & all(n % i for i in range(2, n)):
            c += 1

    return c

In [ ]:
def target(numeri, output, dormi, verboso):
    output.put(conta_numeri_primi(numeri, dormi, verboso))

def conta_numeri_primi_parallelo(numeri, classe, dormi={}, verboso=False, 
                                 indices_or_sections=4):
    output = mp.Queue()
    
    processi = [classe(target=target, args=(n, output, dormi, verboso)) 
                for n in np.array_split(numeri, indices_or_sections)]
    
    for p in processi:
        p.start()
    
    for p in processi:
        p.join()

    return sum([output.get() for p in processi])

In [ ]:
n = conta_numeri_primi(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))
print()

In [ ]:
n = conta_numeri_primi_parallelo(numeri, mp.Process, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
n = conta_numeri_primi_parallelo(numeri, threading.Thread, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi(numeri)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, threading.Thread)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, mp.Process)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi(numeri, dormi={0: 0.25})

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, threading.Thread, dormi={0: 0.25})

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, mp.Process, dormi={0: 0.25})

In [ ]:
def conta_numeri_primi_pool_apply(numeri, dormi={}, verboso=False, 
                            indices_or_sections=4, processes=mp.cpu_count()):
    pool = mp.Pool(processes=processes)
    
    risultati = [pool.apply(conta_numeri_primi, args=(n, dormi, verboso)) 
                 for n in np.array_split(numeri, indices_or_sections)] 
    
    return sum(risultati)

def conta_numeri_primi_pool_apply_async(numeri, dormi={}, verboso=False, 
                            indices_or_sections=4, processes=mp.cpu_count()):
    pool = mp.Pool(processes=processes)
    
    risultati = [pool.apply_async(conta_numeri_primi, args=(n, dormi, verboso)) 
                 for n in np.array_split(numeri, indices_or_sections)] 
    
    return sum([p.get() for p in risultati])

In [ ]:
n = conta_numeri_primi_pool_apply(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
n = conta_numeri_primi_pool_apply_async(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi(numeri)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply(numeri, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply_async(numeri, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply_async(numeri, indices_or_sections=100)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_parallelo(numeri, threading.Thread, dormi={0: 1})

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_parallelo(numeri, mp.Process, dormi={0: 1})

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply(numeri, indices_or_sections=100)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply_async(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply_async(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool(numeri)

In [ ]:
pool = mp.Pool(processes=4)

risultati = [pool.apply_async(conta_numeri_primi, args=(n, {}, False)) 
             for n in np.array_split(numeri, 100)] 
sum([p.get() for p in risultati])

In [ ]:
pool = mp.Pool(processes=4)
results = [pool.apply_async(cube, args=(x,)) for x in range(1,7)]
output = [p.get() for p in results]
print(output)